In [93]:
import geopandas
import rasterio
import glob

In [94]:
# this will go in a loop for each site and each test plot. Now hardcoding
site = "OSBS"
plot_n = '24'
detection_path = './submission/'+site+'_submission.csv'
ras_path = './RS/'+site+'_'+plot_n+'.tif'
raster = rasterio.open(ras_path)

#if using RGB, we have 10 pixels per meter: if we are evaluating on RGB, we probably need to take into account if using w/l
pix_per_meter = 10
#automatically load only boxes within plots boundaries
gdf = geopandas.read_file(
    detection_path,
    bbox=raster.bounds,
)
gtf = geopandas.read_file(
    './submission/'+site+'_ground.csv',
    bbox=raster.bounds,
)

In [95]:
gtf

,WKT,id,geometry
0,"MULTIPOLYGON (((403592.72 3285158.79,403592.72...",1245,"MULTIPOLYGON (((403592.720 3285158.790, 403592..."
1,"MULTIPOLYGON (((403586.41 3285158.74,403586.41...",1246,"MULTIPOLYGON (((403586.410 3285158.740, 403586..."
2,"MULTIPOLYGON (((403593.25 3285160.75,403593.25...",1248,"MULTIPOLYGON (((403593.250 3285160.750, 403593..."
3,"MULTIPOLYGON (((403582.88 3285159.24,403582.88...",1249,"MULTIPOLYGON (((403582.880 3285159.240, 403582..."
4,"MULTIPOLYGON (((403591.95 3285163.15,403591.95...",1250,"MULTIPOLYGON (((403591.950 3285163.150, 403591..."
5,"MULTIPOLYGON (((403600.17 3285164.58,403600.17...",1251,"MULTIPOLYGON (((403600.170 3285164.580, 403600..."
6,"MULTIPOLYGON (((403597.17 3285166.15,403597.17...",1252,"MULTIPOLYGON (((403597.170 3285166.150, 403597..."
7,"MULTIPOLYGON (((403588.87 3285167.7,403588.87 ...",1253,"MULTIPOLYGON (((403588.870 3285167.700, 403588..."
8,"MULTIPOLYGON (((403593.77 3285168.49,403593.77...",1254,"MULTIPOLYGON (((403593.770 3285168.490, 403593..."
9,"MULTIPOLYGON (((403597.04 3285169.98,403597.04...",1256,"MULTIPOLYGON (((403597.040 3285169.980, 403597..."


In [96]:
# turn WTK into coordinates within in the image
gdf_limits = gdf.bounds
gtf_limits = gtf.bounds

xmin = raster.bounds[0]
ymin = raster.bounds[1]
#xmax = raster.bounds[2]
#ymax = raster.bounds[3]

In [97]:
#length
gdf_limits['maxy'] = (gdf_limits['maxy'] - gdf_limits['miny'])*pix_per_meter
gtf_limits['maxy'] = (gtf_limits['maxy'] - gtf_limits['miny'])*pix_per_meter

#width
gdf_limits['maxx'] = (gdf_limits['maxx'] - gdf_limits['minx'])*pix_per_meter
gtf_limits['maxx'] = (gtf_limits['maxx'] - gtf_limits['minx'])*pix_per_meter

# translate coords to 0,0
gdf_limits['minx'] = (gdf_limits['minx'] - xmin) * pix_per_meter
gdf_limits['miny'] = (gdf_limits['miny'] - ymin) * pix_per_meter
gdf_limits.columns = ['minx', 'miny', 'width', 'length']

#same for groundtruth
gtf_limits['minx'] = (gtf_limits['minx'] - xmin) * pix_per_meter
gtf_limits['miny'] = (gtf_limits['miny'] - ymin) * pix_per_meter
gtf_limits.columns = ['minx', 'miny', 'width', 'length']

gdf_limits[gdf_limits < 0] = 0
gtf_limits[gtf_limits < 0] = 0


In [100]:
gdf_limits = gdf_limits.astype(int)
gtf_limits = gtf_limits.astype(int)

In [101]:
#save edges as a new csv file to be fed into the evaluation code
gdf_limits.to_csv('./eval/'+site+'_'+plot_n+'_detections_edges.csv')
gtf_limits.to_csv('./eval/'+site+'_'+plot_n+'_groundtruth_edges.csv')

In [78]:
gdf_limits

,minx,miny,width,length
0,56.840164,84.596312,45.184426,45.184426
1,105.036885,91.051229,39.590164,27.971312
2,111.061475,112.997951,27.540984,22.807377
3,136.881148,116.010246,25.819672,37.008197
4,159.688525,131.071721,35.286885,39.159836
5,141.184426,157.321721,53.360656,37.008197
6,101.594262,133.653689,44.754098,30.983607
7,77.926230,153.018443,28.401639,33.565574
8,100.303279,171.952869,33.995902,22.807377
9,87.393443,39.842213,68.852459,53.790984


In [79]:
gtf_limits

,minx,miny,width,length
0,97.200000,0.000000,38.600000,24.000000
1,34.100000,0.000000,53.400000,37.100000
2,102.500000,17.500000,24.900000,19.200000
3,0.000000,2.400000,28.900000,22.300000
4,89.500000,41.500000,65.800000,43.700000
5,171.700000,55.800000,24.500000,56.900000
6,141.700000,71.500000,28.600000,33.500000
7,58.700000,87.000000,45.500000,33.400000
8,107.700000,94.900000,31.400000,35.600000
9,140.400000,109.800000,26.700000,15.100000
